In [1]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/sacrum/e-commerce-products-search-engine-recommendation")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: tanmay01bhatt
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/sacrum/e-commerce-products-search-engine-recommendation


100%|██████████| 312k/312k [00:00<00:00, 486MB/s]

In [3]:
import pandas as pd

df = pd.read_csv("/content/e-commerce-products-search-engine-recommendation/data.csv")
df.head()

,id,slug,title,imgs,brand,category,vendor,used,address,availability,...,discounted_price,specifications,description,delivery_fee,delivery_details,warranty,warranty_type,average_rating,num_ratings,reviews
0,0,https://www.mega.pk/mobiles_products/23522/Not...,Nothing Phone 1 8GB RAM 256GB Storage Non PTA ...,['https://www.mega.pk/items_images/Nothing+Pho...,NaN,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",NaN,...,NaN,"{'RAM': '8GB', 'Memory quantity': '', 'Interna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
1,1,https://www.mega.pk/mobiles_products/23458/Opp...,Oppo F21 Pro 8GB Ram 128GB Storage 5G PTA Appr...,['https://www.mega.pk/items_images/Oppo+F21+Pr...,OPPO,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",NaN,...,NaN,"{'RAM': '8gb', 'Memory quantity': '', 'Interna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
2,2,https://www.mega.pk/mobiles_products/24393/Tec...,Tecno Spark 10,['https://www.mega.pk/items_images/Tecno+Spark...,Tecno,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,NaN,"{'RAM': '4GB,8GB', 'Memory quantity': '', 'Int...",NaN,NaN,NaN,1 year,NaN,NaN,NaN,[]
3,3,https://www.mega.pk/mobiles_products/24259/Viv...,Vivo V27 5G,['https://www.mega.pk/items_images/Vivo+V27+5G...,Vivo,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,NaN,"{'RAM': '8GB,12GB', 'Memory quantity': '', 'In...",NaN,NaN,NaN,1 year,NaN,NaN,NaN,[]
4,4,https://www.mega.pk/mobiles_products/24204/App...,Apple Iphone 15 Pro Max,['https://www.mega.pk/items_images/Apple+Iphon...,Apple,Mobile,MEGA.PK,0,"Office 11, 12, 14 Basement Ahmed Center, I-8 M...",Coming Soon,...,NaN,"{'RAM': '8GB', 'Memory quantity': '', 'Interna...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


# Pre-Processing

In [4]:
def clean_text(row):
    title = row['title'] if pd.notna(row['title']) else ""
    brand = row['brand'] if pd.notna(row['brand']) else ""
    category = row['category'] if pd.notna(row['category']) else ""
    specs = ' '.join([f"{k}:{v}" for k, v in eval(row['specifications']).items() if v]) if pd.notna(row['specifications']) else ""
    return f"{title} {brand} {category} {specs}".lower()


In [5]:
df['clean_text'] = df.apply(clean_text, axis=1)

In [6]:
df = df.dropna(subset=['clean_text'])  # Drop rows without descriptions

In [9]:
df['clean_text'].iloc[0]

'nothing phone 1 8gb ram 256gb storage non pta 5g black   mobile ram:8gb internal storage space:256gb main camera pixels:50 mp, f/1.9, 24mm (wide), 1/1.56 battery capacity:li-po 4500 mah, non-removable screen size:6.55 inches 5g support:yes finger print:yes display technology:oled, 1b colors, 120hz, hdr10+, 500 nits (typ), 700 nits (peak) display:6.55 inches oled, 1b colors number of colours:1b screen resolution:1080 x 2400 pixels pixel density:402 ppi sd card:yes compatible memory cards:128gb 8gb ram, 256gb 8gb ram, 256gb 12gb ram built-in camera:yes auto focus:yes built-in flash:yes digital zoom (still image):yes maximum resolution (still):1080p 30fps number of cameras:3 front camera resolution:16 mp, f/2.5, (wide), 1/3.1 optical zoom:yes video recorder:yes digital zoom (video):yes maximum numbers of fps when recording:30fps maximum resolution (video):4k 30fps graphics processor:yes processor core type:octa-core processor speed:1x2.5 ghz cortex-a78 & 3x2.4 ghz cortex-a78 chipset:qual

# Create Embeddings

In [10]:
from sentence_transformers import SentenceTransformer

In [11]:
# Load a pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
# Generate embeddings for product descriptions
df['embeddings'] = df['clean_text'].apply(lambda x: model.encode(x))

# Recommendation System

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [14]:
def recommend_products(query, top_k=5):
    # lower case
    query = query.lower()
    # Embed the query
    query_embedding = model.encode(query)

    # Compute cosine similarity with all products
    df['similarity'] = df['embeddings'].apply(lambda x: cosine_similarity([query_embedding], [x]).flatten()[0])

    # Sort products by similarity score
    recommendations = df.sort_values(by='similarity', ascending=False).head(top_k)
    return recommendations[['title', 'brand', 'category', 'similarity','imgs']]

In [17]:
query = "16GB RAM smartphone"
recommendations = recommend_products(query)
recommendations

,title,brand,category,similarity,imgs
524,Samsung Galaxy Tab S8 Ultra 12GB (RAM) + 256GB...,Samsung,Mobile,0.555051,['https://www.czone.com.pk/images/thumbnails-l...
122,Nokia G21 4GB Ram 128GB Storage PTA Approved LTE,Nokia,Mobile,0.550840,['https://www.mega.pk/items_images/Nokia+G21+4...
132,Samsung Galaxy A33 8GB RAM 128GB Storage 5G PT...,Samsung,Mobile,0.529129,['https://www.mega.pk/items_images/Samsung+Gal...
131,Samsung Galaxy A23 6GB Ram 128GB Storage LTE P...,Samsung,Mobile,0.527768,['https://www.mega.pk/items_images/Samsung+Gal...
43,Samsung Galaxy A04 4GB RAM 64GB Storage PTA Ap...,Samsung,Mobile,0.527119,['https://www.mega.pk/items_images/Samsung+Gal...


In [16]:
query = "Apple phone"
recommendations = recommend_products(query)
recommendations

,title,brand,category,similarity,imgs
1564,Apple iPhone 11,NaN,Mobile,0.499544,['https://images.priceoye.pk/apple-iphone-xi-p...
92,Apple iPhone 14 Pro 128GB Storage PTA Approved...,Apple,Mobile,0.493693,['https://www.mega.pk/items_images/Apple+iPhon...
77,Apple iPhone 14 Pro 512GB Storage Single Sim N...,Apple,Mobile,0.493526,['https://www.mega.pk/items_images/Apple+iPhon...
105,Apple iPhone 14 Pro 128GB Storage Physical Sim...,Apple,Mobile,0.493247,['https://www.mega.pk/items_images/Apple+iPhon...
106,Apple iPhone 14 Pro 256GB Storage Physical Sim...,Apple,Mobile,0.493199,['https://www.mega.pk/items_images/Apple+iPhon...


# Save The Embeddings

In [ ]:
# Save the embeddings to a file
df.to_pickle('product_embeddings.pkl')

# # Load the embeddings later
df = pd.read_pickle('product_embeddings.pkl')